## Import required libraries and packages

In [1]:
import os
import yaml

from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_chroma import Chroma

from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from prompt import Prompt

## Load HTML documents

In [2]:
documents = []
for filename in ["excel.aggregationfunction.html", "excel.alloweditrange.html"]:
    html_loader = UnstructuredHTMLLoader(file_path=f"../excel_js_api_docs/{filename}")
    document = html_loader.load()
    documents.append(document[0])

In [3]:
documents

[Document(metadata={'source': '../excel_js_api_docs/excel.aggregationfunction.html'}, page_content='Read in English\n\nNote\n\nAccess to this page requires authorization. You can try signing in or changing directories.\n\nAccess to this page requires authorization. You can try changing directories.\n\nExcel.AggregationFunction enum\n\nReference\n\nAggregation function for the DataPivotHierarchy.\n\nRemarks\n\n[ API set: ExcelApi 1.8 ]\n\nExamples\n\n// Link to full sample: https://raw.githubusercontent.com/OfficeDev/office-js-snippets/prod/samples/excel/38-pivottable/pivottable-filters-and-summaries.yaml\r\n\r\nasync function genericFunctionSwitch(functionType: Excel.AggregationFunction) {\r\n    await Excel.run(async (context) => {\r\n        const pivotTable = context.workbook.worksheets.getActiveWorksheet().pivotTables.getItem("Farm Sales");\r\n        pivotTable.dataHierarchies.load("no-properties-needed");\r\n        await context.sync();\r\n\r\n        pivotTable.dataHierarchies.

## Split Document texts

In [4]:
embeddings = OpenAIEmbeddings(
    api_key=os.environ["OPENAI_API_KEY"], model="text-embedding-3-small"
)

In [5]:
semantic_splitter = SemanticChunker(
    embeddings=embeddings,
    breakpoint_threshold_type="gradient",
    breakpoint_threshold_amount=0.8,
)

In [6]:
chunks = semantic_splitter.split_documents(documents)

## Embed splitted chunks

In [7]:
embedding_model = OpenAIEmbeddings(
    api_key=os.environ["OPENAI_API_KEY"], model="text-embedding-3-small"
)

## Store embeddings in a vector database

In [8]:
vector_store = Chroma.from_documents(documents=chunks, embedding=embedding_model)

## Retriever

In [9]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

## Get sheet state

In [10]:
config_file = "./config/config.yaml"
agent_name = "gpt-4o-mini"
model_name = "gpt-4o-mini"
few_shot_count = 2

In [11]:
def set_agent_configuration(configuration_file_path, agent_name):
    with open(configuration_file_path, mode="r") as file:
        config = yaml.load(file, Loader=yaml.Loader)

    agent_config = config["Agent"]
    return agent_config

In [12]:
agent_config = set_agent_configuration(
    configuration_file_path=config_file, agent_name=agent_name
)
prompt = Prompt(
    agent_configuration=agent_config,
    agent_name=agent_name,
    model_name=model_name,
    few_shot_count=few_shot_count,
)

In [13]:
sheet_state = prompt.get_sheet_state("../JS_Excel_files/EntireShippingCosts.xlsx")

## Get JavaScript code answers

In [14]:
def read_js_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [15]:
task_10_js_code = read_js_file("../JS_Excel_files/task_10.js")
task_11_js_code = read_js_file("../JS_Excel_files/task_11.js")

## Few shot prompt

In [16]:
few_shot_examples = [
    {
        "input": f"Duplicate the first 10 rows in Sheet1 in Sheet2. \n Sheet state: {sheet_state}", 
        "output": f"{task_10_js_code}"
    },
    {
        "input": f"Display only the rows where the distances to Milwaukee are less than 2000 miles. \n Sheet state: {sheet_state}", 
        "output": f"{task_11_js_code}"
    },
]

In [17]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"), 
    ("ai", "{output}"),
])

In [18]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=few_shot_examples,
)

## Final Prompt

In [19]:
user_input = (
    "\n"
    "excel_js_api: {excel_js_api}\n"
    "input: {input}\n"
)

In [20]:
system_message = (
    "You are a JavaScript, TypeScript, and Excel expert.\n"
    "Generate the TypeScript code to manipulate Excel using Excel JavaScript API for each given task.\n"
    "The Excel sheet state is provided.\n"
)

In [21]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        few_shot_prompt,
        ("human", user_input)
    ]
)

In [22]:
print(final_prompt.format(input="What is 5+6?", excel_js_api="HTML docs"))

System: You are a JavaScript, TypeScript, and Excel expert.
Generate the TypeScript code to manipulate Excel using Excel JavaScript API for each given task.
The Excel sheet state is provided.

Human: Duplicate the first 10 rows in Sheet1 in Sheet2. 
 Sheet state: Sheet state: Sheet "Sheet1" has 5 columns (Headers are A: "Customers", B: "Seattle", C: "Milwaukee", D: "Birmingham", E: "Oakland") and 71 rows (1 header row and 70 data rows).
AI: $("#run").on("click", () => tryCatch(run));

async function run() {
  await Excel.run(async (context) => {
    let sheets = context.workbook.worksheets;
    let sheet1 = sheets.getItem("Sheet1");

    sheet1.load("position"); // Load the position of Sheet1

    await context.sync(); // Sync before using loaded properties

    // Add a new sheet
    let sheet2 = sheets.add("Sheet2");
    sheet2.position = sheet1.position; // Insert before Sheet1

    // Copy headers
    let headers = sheet1.getRange("A1:E1");
    sheet2.getRange("A1:E1").copyFrom(hea